<a href="https://colab.research.google.com/github/ivame/DANO_final/blob/main/Important_conversion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def make_group(event: str):
  # Посещение корзины с последующим оформлением заказа
  g1 = ['Посещение корзины с добавленными товарами', 'Ввод промокода']

  # 1е шаги оформления
  g2 = ['Клик на кнопку Перейти к оформлению', 'Выбор радиобаттона eSIM', 'Выбор радиобаттона обычной SIM']

  # Изменение тарифа
  g3 = ['Выбор другого тарифа или номера в корзине']

  # Ввод личных данных
  g4 = ['Ввод контактного номера', 'Ввод имени', 'Ввод email',
        'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)', 'Клик на кнопку Продолжить 2 (Экран выбора региона)']

  # Оформление доставки
  g5 = ['Клик на кнопку Продолжить 3 (Экран выбора способа получения)', 'Клик на кнопку Продолжить 4 (Экран выбора адреса доставки/самовывоза)']

  # Заполнение паспортных данных
  g6 = ['Включение блока паспортных данных', 'Успешное заполнение всех полей паспортных данных']

  # Покупка
  g7 = ['Покупка']

  if event in g1:
    return 1
  elif event in g2:
    return 2
  elif event in g3:
    return 3
  elif event in g4:
    return 4
  elif event in g5:
    return 5
  elif event in g6:
    return 6
  elif event in g7:
    return 7

In [5]:
def good_events(arr):
    ''' Функция убирает те действия, которые повторяются, а также циклы повторяющихся действий. '''
    ans = [arr[0]]
    m = arr[0]
    for i in range(1, len(arr)):
        if arr[i] > arr[i-1] and arr[i] > m:
            ans.append(arr[i])
            m = arr[i]
    return ans

In [6]:
def add_event(line: str):
    ans = ''
    minimum = 1
    maximum = max(line)
    for i in range(int(minimum), int(maximum)+1):
        i = str(i)
        ans = ans + i if ((i in '36') and (i in line)) or (i not in '36') else ans
    return ans


In [7]:
def sessia_name(df_sessions):
  df_sessions = df_sessions[['client_id', 'event_name', 'event_time']]
  df_sessions = df_sessions.sort_values('event_time')
  df_sessions['event_name'] = df_sessions.event_name.apply(lambda x: make_group(x))
  df_sessions = df_sessions.groupby(['client_id'])['event_name'].agg(lambda x: good_events(x.to_list())).reset_index().sort_values('event_name')

  return df_sessions

In [8]:
def seesia_right_name(df_sessions):
  df_sessions_buy = df_sessions.copy()

  # Сессии без покупки
  df_sessions = df_sessions[df_sessions.event_name.apply(lambda x: 7 not in x)]

  # Сессии с покупкой
  df_sessions_buy = df_sessions_buy[df_sessions_buy.event_name.apply(lambda x: 7 in x)]
  df_sessions_buy_copy = df_sessions_buy.copy()

  df_sessions_buy_copy.event_name = df_sessions_buy_copy.event_name.apply(lambda x: x[0:(x.index(7)+1)])
  df_sessions_buy_copy = df_sessions_buy_copy[df_sessions_buy_copy['event_name'].apply(lambda x: len(x) > 0)]
  df_sessions_buy.event_name = df_sessions_buy.event_name.apply(lambda x: x[(x.index(7)+1):])
  df_sessions_buy = df_sessions_buy[df_sessions_buy['event_name'].apply(lambda x: len(x) > 0)]

  # Объединение всех сессий в 1 датафрейм
  df_sessions_buy = pd.concat([df_sessions_buy, df_sessions_buy_copy]).reset_index()
  df_sessions = pd.concat([df_sessions_buy, df_sessions]).reset_index()[['client_id', 'event_name']]

  # Заполение обязательных действий в случаи их пропусков
  df_sessions.event_name = df_sessions.event_name.apply(lambda x: ''.join(str(elem) for elem in x))
  df_sessions['event_name'] = df_sessions.event_name.apply(lambda x: add_event(x))

  return df_sessions

In [9]:
# можно не запускать ульяне
def konversia(vibor, is_ad: str, sex: str, region: str):
  event = ['1', '2', '4', '5', '7']
  count = len(vibor)
  is_ad = 'no' if is_ad == '(not set)' else 'yes'

  print(f'Region: {region}, Gender: {sex}, Marketing: {is_ad}')

  for i in range(1, len(event)):
    cnt = len(vibor[vibor['event_name'].apply(lambda x: event[i] in x)])
    print('\t', event[i-1],' --> ',event[i],' : ',round(cnt/count*100,2),"%", sep='')
    count = cnt

  count = len(vibor)
  print('\t', 1,' --> 7 : ',round(cnt/count*100,2),"%", sep='')

In [10]:
# можно не запускать ульяне
def konversia_all(vibor):
  event = ['1', '2', '4', '5', '7']
  count = len(vibor)


  for i in range(1, len(event)):
    cnt = len(vibor[vibor['event_name'].apply(lambda x: event[i] in x)])
    print('\t', event[i-1],' --> ',event[i],' : ',round(cnt/count*100,2),"%", sep='')
    count = cnt

  count = len(vibor)
  print('\t', 1,' --> 7 : ',round(cnt/count*100,2),"%", sep='')

In [ ]:
viborka = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/for_nelli_conversions_1.csv')

is_ad = '1'
sex = 'Female'
region = 'small_regions'

viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)

konversia(viborka, is_ad, sex, region)

Region: small_regions, Gender: Female, Marketing: yes
	1 --> 2 : 55.49%
	2 --> 4 : 55.37%
	4 --> 5 : 80.61%
	5 --> 7 : 78.48%
	1 --> 7 : 19.44%


In [ ]:
viborka = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/for_nelli_conversions_2.csv')

is_ad = '1'
sex = 'Male'
region = 'small_regions'

viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)

konversia(viborka, is_ad, sex, region)

Region: small_regions, Gender: Male, Marketing: yes
	1 --> 2 : 50.7%
	2 --> 4 : 56.02%
	4 --> 5 : 90.11%
	5 --> 7 : 85.44%
	1 --> 7 : 21.87%


In [ ]:
viborka = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/for_nelli_conversions_3.csv')

is_ad = '1'
sex = 'Female'
region = 'big_regions'

viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)

konversia(viborka, is_ad, sex, region)

Region: big_regions, Gender: Female, Marketing: yes
	1 --> 2 : 49.29%
	2 --> 4 : 53.55%
	4 --> 5 : 87.07%
	5 --> 7 : 87.32%
	1 --> 7 : 20.07%


In [ ]:
viborka = pd.read_csv('/content/drive/MyDrive/Dano/tele2_dataset.csv')


viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)

konversia_all(viborka)

<ipython-input-207-52326cf1b333>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  viborka = pd.read_csv('/content/drive/MyDrive/Dano/tele2_dataset.csv')


	1 --> 2 : 49.47%
	2 --> 4 : 50.0%
	4 --> 5 : 84.66%
	5 --> 7 : 82.41%
	1 --> 7 : 17.26%


In [13]:
# надо ульяне!!!
viborka = pd.read_csv('/content/drive/MyDrive/Dano/tele2_dataset.csv')


viborka = sessia_name(viborka)
viborka = seesia_right_name(viborka)
event_count = []

for i in range(1, 8):
  viborka['d_'+str(i)] = viborka['event_name'].apply(lambda x: x.count(str(i)))
  event_count.append(viborka['d_'+str(i)].sum())

event_count

<ipython-input-13-42afc203e43c>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  viborka = pd.read_csv('/content/drive/MyDrive/Dano/tele2_dataset.csv')


[22503, 11132, 6976, 5566, 4712, 334, 3883]

In [12]:
viborka

,client_id,event_name,d_1,d_2,d_3,d_4,d_5,d_6,d_7
0,775786135.166859,1234567,1,1,1,1,1,1,1
1,1124148272.166868,1234567,1,1,1,1,1,1,1
2,2098039614.166889,1234567,1,1,1,1,1,1,1
3,942281205.166834,1234567,1,1,1,1,1,1,1
4,1886756800.1667705537,1234567,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
22498,1664098714.166409,124,1,1,0,1,0,0,0
22499,277491001.166807,1245,1,1,0,1,1,0,0
22500,1667647084.166765,1245,1,1,0,1,1,0,0
22501,69868234.166935,1245,1,1,0,1,1,0,0


In [21]:
#  (x ∨ y) → (z ≡ x).

for x in range(0, 2):
  for y in range(0, 2):
    for z in range(0, 2):
      if ((x or y) <= (z == x)) == 0:
        print(x, y, z)



0 1 1
1 0 0
1 1 0
